本来我是想看看这几只：
"510050","510310","510500","512000","512560","512600"他们的价格相关性，
后来挑了3只最不相关的一起投：
python -m dingtou.pyramid_v2.pyramid_v2 \
-c 510500,512000,512560 \
-s 20150101 \
-e 20230101 \
-b sh000001 \
-a 500000 \
-m -480 \
-gs 1000 \
-gh 0.01 \
-o 5
效果依然一般，不是说相关就一定会好。结论是。


In [169]:
import akshare as ak
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import talib
import numpy as np
from matplotlib.font_manager import FontProperties
import os
import mplfinance as mpf 

import importlib

from dingtou import utils
importlib.reload(utils)

from dingtou.pyramid_v2 import research4
importlib.reload(research4)

from dingtou.backtest.data_loader import load_fund

%matplotlib inline
# 添加字体
matplotlib.rcParams['axes.unicode_minus']=False # 如果不加入下面这一句，会导致负号无法显示
plt.rcParams["font.sans-serif"] = ["SimHei"]

# 算几个试试先

In [38]:
fund_codes = ["510050","510310","510500","512000","512560","512600"]

dfs  = [load_fund(code)[['code','close']] for code in fund_codes]
dfs[0].columns
dfs[0].index

DatetimeIndex(['2004-12-30', '2004-12-31', '2005-01-07', '2005-01-14',
               '2005-01-21', '2005-01-28', '2005-02-04', '2005-02-18',
               '2005-02-21', '2005-02-22',
               ...
               '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29',
               '2022-12-30', '2022-12-31', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06'],
              dtype='datetime64[ns]', name='date', length=4379, freq=None)

In [43]:

df1s = [df.rename(columns={'close':df.iloc[0].code}) for df in dfs]
df1s

df2s = [df.drop(columns=['code']) for df in df1s]
df2s

[            510050
 date              
 2004-12-30   1.000
 2004-12-31   0.996
 2005-01-07   0.981
 2005-01-14   0.983
 2005-01-21   0.988
 ...            ...
 2022-12-31   3.871
 2023-01-03   3.878
 2023-01-04   3.890
 2023-01-05   3.948
 2023-01-06   3.963
 
 [4379 rows x 1 columns],
             510310
 date              
 2013-03-06  1.0000
 2013-03-08  1.0029
 2013-03-15  1.0002
 2013-03-18  0.9973
 2013-03-19  1.0001
 ...            ...
 2022-12-28  1.8425
 2022-12-29  1.8356
 2022-12-30  1.8426
 2022-12-31  1.8426
 2023-01-03  1.8504
 
 [2395 rows x 1 columns],
             510500
 date              
 2013-02-06  1.0000
 2013-02-08  1.0022
 2013-02-22  0.9954
 2013-03-01  1.0135
 2013-03-08  0.9980
 ...            ...
 2022-12-28  1.8894
 2022-12-29  1.8879
 2022-12-30  1.8901
 2022-12-31  1.8900
 2023-01-03  1.9193
 
 [2399 rows x 1 columns],
             512000
 date              
 2016-08-30  1.0000
 2016-08-31  1.0011
 2016-09-01  0.9988
 2016-09-02  1.0000
 2016-09-07  1.0

In [45]:
df = pd.concat(df2s, axis=1)

df.corr()

,510050,510310,510500,512000,512560,512600
510050,1.000000,0.979907,0.704513,0.573869,0.651437,0.884177
510310,0.979907,1.000000,0.780306,0.627344,0.825118,0.915443
510500,0.704513,0.780306,1.000000,0.718084,0.916627,0.390601
512000,0.573869,0.627344,0.718084,1.000000,0.576537,0.442561
512560,0.651437,0.825118,0.916627,0.576537,1.000000,0.846645
512600,0.884177,0.915443,0.390601,0.442561,0.846645,1.000000


# 算所有ETF的相关性

算上市3年的ETF

In [170]:
data = research4.filter_data(3)
dfs = [ v for k,v in data.items()]
dfs

print("一共%d个ETF基金数据:",len(data))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 379/379 [00:01<00:00, 261.42it/s]
588460:科创50增强ETF , 上市时间20221201小于3年(0年/1月/8天)
588400:双创50ETF , 上市时间20210625小于3年(1年/6月/15天)
588390:科创创业50ETF , 上市时间20210819小于3年(1年/4月/21天)
588380:创50ETF , 上市时间20210629小于3年(1年/6月/11天)
588370:科创50增强策略ETF , 上市时间20221201小于3年(0年/1月/8天)
588360:科创创业ETF , 上市时间20210629小于3年(1年/6月/11天)
588350:双创50ETF基金 , 上市时间20221026小于3年(0年/2月/14天)
588330:双创龙头ETF , 上市时间20210629小于3年(1年/6月/11天)
588320:双创50增强ETF , 上市时间20221228小于3年(0年/0月/12天)
588310:双创ETF基金 , 上市时间20211122小于3年(1年/1月/18天)
588300:双创ETF , 上市时间20210625小于3年(1年/6月/15天)
588290:科创板芯片ETF , 上市时间20220930小于3年(0年/3月/10天)
588280:科创50ETF华安 , 上市时间20211012小于3年(1年/2月/28天)
588260:科创信息ETF , 上市时间20220518小于3年(0年/7月/22天)
588200:科创芯片ETF , 上市时间20220930小于3年(0年/3月/10天)
588180:科创50ETF基金 , 上市时间20210624小于3年(1年/6月/16天)
588160:科创板新材料ETF , 上市时间20220930小于3年(0年/3月/10天)
588150:科创龙头ETF , 上市时间20211210小于3年(1年/0月/30天)
58

516180:光伏ETF平安 , 上市时间20210209小于3年(1年/11月/0天)
516160:新能源ETF , 上市时间20210122小于3年(1年/11月/18天)
516150:稀土ETF基金 , 上市时间20210309小于3年(1年/10月/0天)
516130:消费龙头ETF , 上市时间20210928小于3年(1年/3月/12天)
516120:化工50ETF , 上市时间20210301小于3年(1年/10月/8天)
516110:汽车ETF , 上市时间20210407小于3年(1年/9月/2天)
516100:互联网金融ETF , 上市时间20210713小于3年(1年/5月/27天)
516090:新能源ETF易方达 , 上市时间20210311小于3年(1年/9月/29天)
516080:创新药ETF易方达 , 上市时间20210203小于3年(1年/11月/6天)
516070:碳中和50ETF , 上市时间20210415小于3年(1年/8月/25天)
516060:创新药产业ETF , 上市时间20210209小于3年(1年/11月/0天)
516050:科技龙头ETF , 上市时间20210121小于3年(1年/11月/19天)
516020:化工ETF , 上市时间20210226小于3年(1年/10月/14天)
516010:游戏ETF , 上市时间20210225小于3年(1年/10月/15天)
516000:大数据50ETF , 上市时间20210209小于3年(1年/11月/0天)
515960:医药健康ETF , 上市时间20200424小于3年(2年/8月/16天)
515950:医药龙头ETF , 上市时间20200316小于3年(2年/9月/24天)
515930:永赢沪深300ETF , 上市时间20200413小于3年(2年/8月/27天)
515920:智能消费ETF , 上市时间20201230小于3年(2年/0月/10天)
515910:质量ETF , 上市时间20210121小于3年(1年/11月/19天)
515890:红利ETF博时 , 上市时间20200320小于3年(2年/9月/20天)
515850:证券龙头ETF , 上市时间20200121小于3年(2年/11月/19天)
515

一共%d个ETF基金数据: 142


In [171]:

df1s = [df[['code','close']] for df in dfs]
df1s

df2s = [df.rename(columns={'close':df.iloc[0].code}) for df in df1s]
df2s

df3s = [df.drop(columns=['code']) for df in df2s]
df3s

df4 = pd.concat(df3s,axis=1)
df4


df_corr = df4.corr()
df_corr

,515990,515980,515900,515880,515870,515860,515810,515800,515750,515700,...,510130,510120,510110,510100,510090,510060,510050,510030,510020,510010
515990,1.000000,-0.036433,0.911249,-0.390860,0.881158,0.319521,0.836255,0.795304,0.532085,0.905593,...,0.972199,0.768351,0.507800,0.681206,0.908098,0.821661,0.435084,0.927047,0.486740,0.900021
515980,-0.036433,1.000000,-0.249534,0.849480,0.358395,0.894609,0.373876,0.464781,0.800777,-0.208647,...,0.068486,0.423100,0.599098,0.471575,0.158194,0.304958,0.650139,0.054918,0.653144,0.174778
515900,0.911249,-0.249534,1.000000,-0.461040,0.695602,0.086726,0.612775,0.556748,0.335088,0.930014,...,0.856763,0.528298,0.192749,0.404285,0.732722,0.605496,0.107709,0.790815,0.176219,0.714460
515880,-0.390860,0.849480,-0.461040,1.000000,-0.029670,0.527510,-0.045212,0.035997,0.420272,-0.479065,...,-0.267435,-0.011828,0.240803,0.048918,-0.225314,-0.083210,0.279840,-0.292164,0.260700,-0.215085
515870,0.881158,0.358395,0.695602,-0.029670,1.000000,0.649764,0.957950,0.952456,0.817023,0.707840,...,0.931610,0.921692,0.759418,0.874008,0.931279,0.904564,0.729428,0.887462,0.773891,0.935199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510060,0.821661,0.304958,0.605496,-0.083210,0.904564,0.647420,0.909941,0.930268,0.727443,0.552371,...,0.928862,0.937734,0.980549,0.923574,0.912446,1.000000,0.950883,0.938828,0.936488,0.966838
510050,0.435084,0.650139,0.107709,0.279840,0.729428,0.823330,0.791014,0.857174,0.794975,0.133814,...,0.926666,0.950366,0.983242,0.929218,0.970993,0.950883,1.000000,0.976995,0.985385,0.979785
510030,0.927047,0.054918,0.790815,-0.292164,0.887462,0.499204,0.897907,0.867063,0.613236,0.723043,...,0.939357,0.938389,0.960732,0.839475,0.988367,0.938828,0.976995,1.000000,0.956241,0.983410
510020,0.486740,0.653144,0.176219,0.260700,0.773891,0.854527,0.840385,0.887639,0.831069,0.171792,...,0.931136,0.953779,0.957606,0.952829,0.964598,0.936488,0.985385,0.956241,1.000000,0.965695


In [175]:
# 找出相关性非常低的那些ETF
df_corr_select = df_corr[df_corr<0]

In [186]:
import pandas as pd
import numpy as np
df_valid = pd.DataFrame()
cols = df_corr.columns

df_corr_select = df_corr[df_corr<-0.5]

for index, row in df_corr_select.iterrows():
        for i,corr in enumerate(row):
            if not np.isnan(corr):
                df_valid = df_valid.append({'code1':index,'code2':cols[i],'corr':corr},ignore_index=True)  


print("相关性低的ETF对有：%d个/%d个, %.3f%%" % (
    len(df_valid),
    len(cols)*len(cols),
    len(df_valid)*100/(len(cols)*len(cols))))

# 合并code和code2
df_codes = df_valid.code1.append(df_valid.code2)
print(len(df_codes))
print("合并后有：",len(df_codes.unique()))
print(df_codes.unique())
df_valid


相关性低的ETF对有：90个/20164个, 0.446%
180
合并后有： 31
['515980' '515900' '515880' '515700' '515680' '515600' '515300' '515180'
 '515080' '512980' '512960' '512950' '512890' '512810' '512680' '512670'
 '512660' '512590' '512580' '512560' '512400' '512260' '512200' '512190'
 '512100' '512040' '510900' '510880' '510590' '510410' '510170']


,code1,code2,corr
0,515980,512890,-0.566388
1,515980,510410,-0.507956
2,515900,512980,-0.584940
3,515900,512200,-0.513538
4,515900,510900,-0.639787
...,...,...,...
85,510880,515880,-0.600590
86,510590,510900,-0.518183
87,510410,515980,-0.507956
88,510410,515880,-0.616816


In [196]:
# 剩下的31个之间再做一个corr，找出跟别人相关性高的里最少得
df5 = df4[df_codes.unique()]
df6 = df5.corr()[df5.corr()>0.9].count()
df6[df6<10]

515980    1
515880    1
515300    5
512980    1
512200    1
510900    1
510880    5
dtype: int64

In [166]:
df_v1 = df_valid[df_valid!=1.0]
df_v2 = df_v1.dropna()
df_v3 = df_v2.sort_values(by='corr',ascending=False)
df_v4 = df_v3.drop_duplicates(subset=['corr'])
df_v5 = df_v4[df_v4['corr']>0.99]
print("相关性大于0.99的 ETF对 有：", len(df_v5))
df_v5
# len(df_v5.code1.unique())
# df_valid.sort_values(by='corr')
# df_codes = df_valid.code1.append(df_valid.code2)
# len(df_codes.unique())

相关性大于0.99的 ETF对 有： 229


,code1,code2,corr
1,518880,518800,0.999934
2125,515380,515330,0.999926
4202,512950,512960,0.999883
13335,510330,515380,0.999868
13338,510330,515330,0.999863
...,...,...,...
3933,512970,515660,0.990174
10190,512090,515350,0.990124
11726,510600,515380,0.990034
6643,512660,512670,0.990021
